In [1]:
using Distributions, TimeIt, ProgressMeter, PyPlot
include("/home/postdoc/dsen/Desktop/G-ZZ/jl_files/samplers.jl")

GibbsHMC (generic function with 1 method)

In [11]:
d, Nobs = 20, 50
pX, pξ = 1e-1, 1e-1
X = sprandn(d, Nobs, pX)
X[1,:] = ones(Nobs)
ξ_true = sprandn(d,pξ)
ξ_true[1] = randn()
y = Float64.([rand(Binomial(1, 1/(1+exp(-ξ_true'X[:,j]))), 1)[1] for j in 1:Nobs])
mb_size = 5
σ02 = 1

prior = SS_prior(d, σ02);

In [12]:
my_ll = ll_logistic_sp(X,y);
my_model = model(my_ll, prior);

In [13]:
# Sub-sampling with weights and no CV

prob_het = 0.98
gs = Array{mbsampler}(d)
gs[1] = umbsampler(0, Nobs, mb_size)

@showprogress for i in 2:d
    weights_het = abs.(X[i,:])./sum(abs.(X[i,:]))
    gs[i] = spwumbsampler(Nobs, mb_size, weights_het, prob_het)
end
gs_list = mbsampler_list(d,gs);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


In [14]:
A_xi = eye(d)
A_hyp = eye(hyperparam_size(prior))

opf = projopf(A_xi, A_hyp, 1000)
opt = maxa_opt(10^6, 2*10^5)
outp = outputscheduler(opf,opt)
bb = linear_bound(my_model.ll, my_model.pr, gs_list)
mstate = zz_state(d)
update_bound(bb, my_ll, prior, gs_list, mstate);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


In [15]:
print("Initial speed = ", opf.alpha_skeleton[:,opf.tcounter+1])

Initial speed = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [16]:
adapt_speed = "by_var"
L = 1
my_zz_sampler = zz_sampler(0, gs_list, bb, L, adapt_speed)
hyper_sampler = block_gibbs_sampler(1e2)
blocksampler = Array{msampler}(2)
blocksampler[1] = my_zz_sampler
blocksampler[2] = hyper_sampler;

In [ ]:
start = time()
ZZ_block_sample(my_model, outp, blocksampler, mstate)
print("Total time: ", round((time()-start)/60, 2), " mins")

10 percent attempts in 0.51 min; zz bounces = 6162, hyp bounces = 43067, total time of process = 434.690 
20 percent attempts in 1.04 min; zz bounces = 12369, hyp bounces = 87522, total time of process = 882.209 
Trimmed after 200000 bouncing attempts 
30 percent attempts in 1.57 min; zz bounces = 18103, hyp bounces = 132519, total time of process = 1332.246 
40 percent attempts in 2.14 min; zz bounces = 24210, hyp bounces = 177599, total time of process = 1780.641 
50 percent attempts in 2.71 min; zz bounces = 30032, hyp bounces = 222646, total time of process = 2229.166 


In [ ]:
fig = figure("pyplot_barplot",figsize=(8,3))
for i in 1:d 
    plot((outp.opf.alpha_skeleton[i,:]))
end
title("Speeds")
xlabel("bounce")
grid(true)

In [ ]:
discard = 0
n_samples = 10^4
xi_samples = extract_samples(outp.opf.xi_skeleton[:,discard+1:end], 
                             outp.opf.bt_skeleton[:,discard+1:end], 
                             outp.opf.bt_skeleton[end]/n_samples,
                             "linear");
hyper_samples = extract_samples(outp.opf.hyper_skeleton[:,discard+1:end], 
                                outp.opf.bt_skeleton[:,discard+1:end], 
                                outp.opf.bt_skeleton[end]/n_samples, 
                                "constant");

In [ ]:
fig = figure("pyplot_barplot",figsize=(7,2.5))
plot([compute_configT(my_model, xi_samples[:,1:end-1], hyper_samples[:,1:end-1], k) for k in 1:d], "o")
grid(true)

In [ ]:
mean(xi_samples,2)

In [ ]:
mstate.mu